🏥 Dia-Agent: Privacy-First Clinical Support System

Diabetes Care with Human-in-the-Loop Safety

Step 1: Environment Setup & Installation

In [1]:
# @title 📦 1. Install Dependencies
# Run this cell to install the required packages
!pip install -q google-adk google-genai python-dotenv pandas

import sys
import os
import asyncio
import pandas as pd

# Validate installation
if "google.adk" in sys.modules:
    print("✅ Google ADK Installed Successfully!")
else:
    print("⚠️ Please restart the kernel if imports fail.")


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


⚠️ Please restart the kernel if imports fail.


Step 2: Configure Security (API Key)

In [6]:
import os
# Import the library you installed to read the .env file
from dotenv import load_dotenv 

# 1. Load environment variables from .env file
load_dotenv()

# 2. Try loading from Google Colab Secrets (for Cloud)
try:
    from google.colab import userdata
    colab_key = userdata.get('GOOGLE_API_KEY')
    if colab_key:
        os.environ["GOOGLE_API_KEY"] = colab_key
        print("✅ API Key loaded from Colab Secrets.")
except ImportError:
    pass

# 3. Verify the key is loaded (works for both Local and Cloud)
if os.environ.get("GOOGLE_API_KEY"):
    print("✅ API Key loaded securely from environment.")
else:
    print("⚠️ WARNING: API Key not found. Please check your .env file or Colab Secrets.")

✅ API Key loaded securely from environment.


Step 3: Define Clinical Logic (Custom Tools)

In [32]:
%%writefile diabetes_tools.py
from google.adk.tools.tool_context import ToolContext

def calculate_clinical_risk(glucose: float, bmi: float, age: int, hypertension: int) -> dict:
    """
    Analyzes patient metrics to determine Diabetes Risk Level.
    Derived from Random Forest analysis of clinical data.
    """
    risk_score = 0
    risk_factors = []

    # Clinical Rules
    if glucose > 125:
        risk_score += 3
        risk_factors.append("High Glucose (>125 mg/dL)")
    elif glucose > 100:
        risk_score += 1
        risk_factors.append("Elevated Glucose")

    if bmi > 30:
        risk_score += 2
        risk_factors.append("Obesity (BMI > 30)")
    
    if hypertension == 1:
        risk_score += 1
        risk_factors.append("Hypertension")

    if age > 45:
        risk_score += 1

    # Risk Classification
    if risk_score >= 3:
        return {
            "risk_level": "HIGH", 
            "score": risk_score, 
            "factors": risk_factors, 
            "advice": "Immediate medical consultation required."
        }
    elif risk_score >= 1:
        return {
            "risk_level": "MODERATE", 
            "score": risk_score, 
            "factors": risk_factors, 
            "advice": "Lifestyle modification recommended."
        }
    else:
        return {
            "risk_level": "LOW", 
            "score": risk_score, 
            "factors": risk_factors, 
            "advice": "Routine checkup schedule."
        }

def notify_doctor_critical(risk_level: str, patient_id: str, tool_context: ToolContext) -> dict:
    """
    Safety Protocol: Pauses execution if Risk is HIGH to get Human Doctor Approval.
    """
    # 1. Auto-approve low/moderate risk
    if risk_level != "HIGH":
        return {"status": "logged", "message": f"Risk is {risk_level}. Logged to internal system."}

    # 2. First Call: Trigger Pause
    if not tool_context.tool_confirmation:
        print(f"\n[SYSTEM ALERT] 🛑 HIGH RISK DETECTED for {patient_id}. Pausing for Doctor Approval...")
        
        tool_context.request_confirmation(
            hint=f"Approve Critical Alert for {patient_id}?",
            payload={"patient_id": patient_id, "risk": risk_level}
        )
        return {"status": "pending", "message": "Waiting for human doctor approval."}

    # 3. Resumed Call: Process Decision
    if tool_context.tool_confirmation.confirmed:
        return {"status": "sent", "message": f"✅ ALERT SENT to Dr. Smith for {patient_id}."}
    else:
        return {"status": "rejected", "message": "❌ Alert cancelled by supervisor."}

Overwriting diabetes_tools.py


The Main Application (Agents & Simulation)

4a: Agent Configuration (The Brain)

In [34]:
# @title 🧠 4a. Define Agents & Pipeline
from google.adk.agents import LlmAgent, SequentialAgent
from google.adk.models.google_llm import Gemini
from google.adk.apps.app import App, ResumabilityConfig
from google.adk.plugins.logging_plugin import LoggingPlugin
from google.genai import types

# Import tools
from diabetes_tools import calculate_clinical_risk, notify_doctor_critical

# Configuration
MODEL_NAME = "gemini-2.0-flash"
retry_config = types.HttpRetryOptions(attempts=3)

# --- AGENT 1: Clinical Analyst ---
analyst_agent = LlmAgent(
    name="ClinicalAnalyst",
    model=Gemini(model=MODEL_NAME, retry_options=retry_config),
    instruction="""You are an expert Clinical Analyst.
    1. Analyze patient metrics.
    2. Use `calculate_clinical_risk`.
    3. CRITICAL: If risk is found, you MUST use `notify_doctor_critical`.
    4. Output a structured clinical summary.""",
    tools=[calculate_clinical_risk, notify_doctor_critical],
    output_key="clinical_summary"
)

# --- AGENT 2: Lifestyle Coach ---
coach_agent = LlmAgent(
    name="LifestyleCoach",
    model=Gemini(model=MODEL_NAME, retry_options=retry_config),
    instruction="""You are a Lifestyle Coach.
    1. Read the clinical summary: {clinical_summary}
    2. Use `load_memory` to check patient preferences.
    3. Create a plan that respects their preferences AND addresses medical risk.""",
    # Note: load_memory tool is added by the Runner automatically if we configure memory service
    tools=[] 
)

# --- PIPELINE ---
dia_pipeline = SequentialAgent(
    name="DiaCarePipeline",
    sub_agents=[analyst_agent, coach_agent]
)

# --- APP WRAPPER ---
dia_app = App(
    name="DiaCareApp",
    root_agent=dia_pipeline,
    resumability_config=ResumabilityConfig(is_resumable=True),
    plugins=[LoggingPlugin()]
)

print("✅ Agents & Pipeline Configured.")

✅ Agents & Pipeline Configured.


C:\Users\preet\AppData\Local\Temp\ipykernel_27216\3877780493.py:50: UserWarning: [EXPERIMENTAL] ResumabilityConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  resumability_config=ResumabilityConfig(is_resumable=True),


4b: Initialize Services (The Engine)

In [35]:
# @title ⚙️ 4b. Initialize Runner & Services
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory

# Initialize Services
session_service = InMemorySessionService()
memory_service = InMemoryMemoryService()

# Initialize Runner
runner = Runner(
    app=dia_app,
    session_service=session_service,
    memory_service=memory_service
)

# Add load_memory tool to coach (Dynamic addition)
coach_agent.tools.append(load_memory)

print("✅ Engine Started. Runner is ready.")

App name mismatch detected. The runner is configured with app name "DiaCareApp", but the root agent was loaded from "E:\D_volume\Preethi\OneDrive\Desktop\desktopfiles\Kaggle Agentic AI 5 day course\diabetes_capstone\.venv\Lib\site-packages\google\adk\agents", which implies app name "agents".


✅ Engine Started. Runner is ready.


4c: Seeding Memory (The Setup)

In [36]:
# @title 💾 4c. Seed Long-Term Memory
# We simulate a past visit to teach the agent about the patient
history_session_id = "past_visit_001"
user_id = "patient_001"

# 1. Create Session
await session_service.create_session(
    app_name="DiaCareApp", user_id=user_id, session_id=history_session_id
)

# 2. Input History
history_input = "Patient Update: Glucose 90. Note: Patient is strictly VEGAN and enjoys SWIMMING."
print(f"📝 Injecting History: {history_input}")

async for event in runner.run_async(
    user_id=user_id, 
    session_id=history_session_id, 
    new_message=types.Content(role="user", parts=[types.Part(text=history_input)])
):
    pass 

# 3. Save to Memory
history_session = await session_service.get_session(
    app_name="DiaCareApp", user_id=user_id, session_id=history_session_id
)
await memory_service.add_session_to_memory(history_session)
print("✅ Patient History saved to Memory.")

📝 Injecting History: Patient Update: Glucose 90. Note: Patient is strictly VEGAN and enjoys SWIMMING.
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-ed65c4fb-7c68-4488-bf62-1b25a4cf913c
[logging_plugin]    Session ID: past_visit_001
[logging_plugin]    User ID: patient_001
[logging_plugin]    App Name: DiaCareApp
[logging_plugin]    Root Agent: DiaCarePipeline
[logging_plugin]    User Content: text: 'Patient Update: Glucose 90. Note: Patient is strictly VEGAN and enjoys SWIMMING.'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-ed65c4fb-7c68-4488-bf62-1b25a4cf913c
[logging_plugin]    Starting Agent: DiaCarePipeline
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: DiaCarePipeline
[logging_plugin]    Invocation ID: e-ed65c4fb-7c68-4488-bf62-1b25a4cf913c
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 5fb36cb4-a9b5-4214-81e9-d99943494518
[logging_plugin]    Author: DiaCarePipeline
[logging_plugin]  

4d: Live Analysis & Pause (The Conflict)

In [37]:
# @title 🚨 4d. Live Diagnosis (Triggers Pause)
# High Risk Data
user_query = "Patient ID: 001. New Vitals: Glucose 145 mg/dL, BMI 31.0, Age 55, Hypertension: Yes."
session_id = "live_session_01"

print(f"👤 [Input]: {user_query}")
print("⏳ Agent is thinking...\n")

# Create Session
await session_service.create_session(
    app_name="DiaCareApp", user_id=user_id, session_id=session_id
)

# Variable to store the pause state for the next cell
pause_state = {} 

async for event in runner.run_async(
    user_id=user_id, 
    session_id=session_id, 
    new_message=types.Content(role="user", parts=[types.Part(text=user_query)])
):
    # Pretty Print the Agent's Thought Process
    if event.is_final_response() and event.content:
        print(f"\n🤖 [Analyst]: {event.content.parts[0].text}")
    
    # Detect Pause Request
    if event.content and event.content.parts:
        for part in event.content.parts:
            if part.function_call and part.function_call.name == "adk_request_confirmation":
                print("\n" + "="*50)
                print("🛑 SYSTEM PAUSED: Doctor Approval Required")
                print(f"   Reason: High Risk Detected for {user_id}")
                print("="*50 + "\n")
                # Save state for the next cell
                pause_state = {
                    "approval_id": part.function_call.id, 
                    "inv_id": event.invocation_id
                }

👤 [Input]: Patient ID: 001. New Vitals: Glucose 145 mg/dL, BMI 31.0, Age 55, Hypertension: Yes.
⏳ Agent is thinking...

[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-a306e595-a914-4d3d-82f9-a5f657ec120a
[logging_plugin]    Session ID: live_session_01
[logging_plugin]    User ID: patient_001
[logging_plugin]    App Name: DiaCareApp
[logging_plugin]    Root Agent: DiaCarePipeline
[logging_plugin]    User Content: text: 'Patient ID: 001. New Vitals: Glucose 145 mg/dL, BMI 31.0, Age 55, Hypertension: Yes.'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-a306e595-a914-4d3d-82f9-a5f657ec120a
[logging_plugin]    Starting Agent: DiaCarePipeline
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: DiaCarePipeline
[logging_plugin]    Invocation ID: e-a306e595-a914-4d3d-82f9-a5f657ec120a
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 65c77b5e-d47b-438b-b207-ea6c136836d5
[logging_plugin]    Author: DiaCarePipe

Human Approval (The Resolution)

In [38]:
# @title 👨‍⚕️ 4e. Doctor Approval & Resume
if not pause_state:
    print("⚠️ No pause detected. Run Cell 4d first.")
else:
    print("👨‍⚕️ Doctor clicks [APPROVE] button on dashboard...")
    print("▶️  Resuming Agent Workflow...\n")
    
    # Create Approval Signal
    confirmation = types.FunctionResponse(
        id=pause_state["approval_id"], 
        name="adk_request_confirmation", 
        response={"confirmed": True} 
    )
    
    # Resume
    async for event in runner.run_async(
        user_id=user_id,
        session_id=session_id,
        new_message=types.Content(role="user", parts=[types.Part(function_response=confirmation)]),
        invocation_id=pause_state["inv_id"] # This connects back to the paused state
    ):
         if event.is_final_response() and event.content:
             print(f"\n🤖 [Coach]: {event.content.parts[0].text}")

    print("\n✅ Case Closed.")

👨‍⚕️ Doctor clicks [APPROVE] button on dashboard...
▶️  Resuming Agent Workflow...

[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-a306e595-a914-4d3d-82f9-a5f657ec120a
[logging_plugin]    Session ID: live_session_01
[logging_plugin]    User ID: patient_001
[logging_plugin]    App Name: DiaCareApp
[logging_plugin]    Root Agent: DiaCarePipeline
[logging_plugin]    User Content: function_response: adk_request_confirmation
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-a306e595-a914-4d3d-82f9-a5f657ec120a
[logging_plugin]    Starting Agent: DiaCarePipeline
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: DiaCarePipeline
[logging_plugin]    Invocation ID: e-a306e595-a914-4d3d-82f9-a5f657ec120a
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: ClinicalAnalyst
[logging_plugin]    Invocation ID: e-a306e595-a914-4d3d-82f9-a5f657ec120a
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: noti